In [1]:
import os
import numpy as np
from skimage import io
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim
import pandas as pd
import numpy as np
from skimage import img_as_float
from tqdm import tqdm

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
num_epochs = 100
batch_size = 300
learning_rate = 0.001
accuracy_training = []

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
im_size = 98
#kernel set = 3 for 96 kernel set = 7 for 98
kernel_set = 7
#144 for 96, 150 for 98
final_feature = 150
default_path= '/home/DL-based-Tumor-Classification/Datasets/Newest_case/1D_sorted/Non-oral+EsophagealSCC'

True
0


In [5]:
for i in range(0,10):
    print("fold " + str(i))
    train_csv_path = default_path+ '/img_fold' + str(i) + '/train/labels_train.csv'
    train_root_path = default_path+ '/img_fold' + str(i) + '/train'
    test_csv_path = default_path+ '/img_fold' + str(i) + '/test/labels_test.csv'
    test_root_path = default_path+ '/img_fold' + str(i) + '/test'
    test_label_path =  default_path+ '/img_fold' + str(i) + '/test/test_label_run_th1_test_multi_relu.csv'
    predicted_label_path = default_path+ '/img_fold' + str(i) + '/test/predicted_label_run_th1_test_multi_relu.csv'
    model_path = default_path+ '/img_fold' + str(i) + '/network_modified_multirelu.pth'


    class TumorDatasetTrain(Dataset):
        
        def __init__(self, csv_file , root_dir, transform=None):
            self.labels_frame = np.array(pd.read_csv(csv_file, skiprows=1, sep=',', header=None)).astype(np.int)
            self.root_dir = root_dir
            self.transform = transform
        
        def __len__(self):
            return len(self.labels_frame)
        
        def __getitem__(self, idx):
            img_name = str(idx)+'.png'
            img_path = os.path.join(self.root_dir, img_name)
            """
                !!!Pay attention!!!
                The image size is set here
                """
            img = np.empty(shape=(1, im_size * im_size))
            img[0, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
            label = np.array([self.labels_frame[idx,1]-1])
            train_sample = {'image': img, 'label': label}
            
            if self.transform:
                train_sample = self.transform(train_sample)
            return train_sample


    class TumorDatasetTest(Dataset):
        
        def __init__(self, csv_file , root_dir, transform=None):
            self.labels_frame = np.array(pd.read_csv(csv_file, skiprows= 1, sep=',', header= None)).astype(np.int)
            self.root_dir = root_dir
            self.transform = transform
        
        def __len__(self):
            return len(self.labels_frame)
        
        def __getitem__(self, idx):
            img_name = str(idx) + '.png'
            img_path = os.path.join(self.root_dir, img_name)
            """
                !!!Pay attention!!!
                The image size is set here
                """
            img = np.empty(shape=(1, im_size * im_size))
            img[0, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
            label = np.array([self.labels_frame[idx, 1]-1])
            test_sample = {'image': img, 'label': label}
            
            if self.transform:
                test_sample = self.transform(test_sample)
            return test_sample


    class ToTensor(object):
        
        def __call__(self, sample):
            image, labels = sample['image'], sample['label']
            return {'image': torch.from_numpy(image), 'label': torch.LongTensor(labels)}


    train_dataset = TumorDatasetTrain(csv_file=train_csv_path, root_dir=train_root_path,
                                      transform=transforms.Compose([ToTensor()]))
    test_dataset = TumorDatasetTest(csv_file=test_csv_path, root_dir=test_root_path, 
                                    transform=transforms.Compose([ToTensor()]))
    dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)


    class Net(nn.Module):

        def __init__(self, num_of_classes):
            super(Net, self).__init__()
            # input image channel, output channels, kernel size square convolution
            # kernel
            # input size = 9216/9604 output size = 9216/9600
            self.conv1 = nn.Conv1d(1, 64, kernel_size=kernel_set, padding=1)
            self.bn1 = nn.BatchNorm1d(64)
            self.relu1 = nn.ReLU(inplace = False)
            # input size = 14//2304/2400 output size = 6//2304/2400
            self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm1d(128)
            self.relu2 = nn.ReLU(inplace = False)
            # input size = 6//576/600, output size = 3//576/600
            self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
            self.bn3 = nn.BatchNorm1d(256)
            self.relu3 = nn.ReLU(inplace = False)
            self.drop = nn.Dropout(p=0.25, inplace=False)
            self.vp = nn.MaxPool1d(kernel_size=4, padding=0, stride=4)
            # an affine operation: y = Wx + b
#           144 for 96, 150 for 98
            self.fc1 = nn.Linear(256*final_feature, 1024)
            self.fc2 = nn.Linear(1024, 512)
            self.fc3 = nn.Linear(512, 256)
            self.fc4 = nn.Linear(256, num_of_classes)


        def forward(self, x):
            in_size = x.size(0)
            x = self.conv1(x)
            x = self.relu1(self.bn1(self.vp(x)))
            x = self.relu2(self.bn2(self.vp(self.conv2(x))))
            x = self.relu3(self.bn3(self.vp(self.conv3(x))))
            x = self.drop(x)
            x = x.view(in_size, -1)
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.fc3(x)
            x = self.fc4(x)
            return x



    net = Net(num_of_classes=6)
    net = net.double()
    net = nn.DataParallel(net)
    net.cuda()
    print('using gpu #', torch.cuda.current_device())

    running_loss = []
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate)
    
    net.train()
    
    for epoch in tqdm(range(num_epochs)):
        running_loss_tmp = 0.0
        for i, sample in enumerate(dataloader_train):
            # images = Variable(image).cuda()
            images = Variable(sample['image']).cuda()
            # print("SEE THIS::: " , images.shape)
            # labels = Variable(labels).cuda()
            labels = Variable(sample['label']).cuda()
            labels = labels.view(-1)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss_tmp += loss.data
        print('epoch', epoch, ':loss is', running_loss_tmp)
        running_loss.append(running_loss_tmp)
        if (epoch > 3) and (abs(running_loss[epoch] - running_loss[epoch-1]) <= 0.0001) and (abs(running_loss[epoch - 1] - running_loss[epoch - 2]) <= 0.0001):
            break
    print('training finished')

    torch.save(net.state_dict(), model_path)
#     state_dict = torch.load(model_path)
#     net.load_state_dict(state_dict)
    
    
    correct = 0
    total = 0
    predicted_save = np.array([])
    test_label_save = np.array([])
    
    net.eval()
    
    for ii, test_sample in enumerate(dataloader_test):
        test_imgs = Variable(test_sample['image']).cuda()
#         print("SEE THIS::: " , test_imgs.shape)
        test_label = Variable(test_sample['label']).cuda()
#         print(test_label.shape)
        outputs = net(test_imgs)
#         print("output: ", outputs.shape)
        _, predicted = torch.max(outputs.data, 1)
#         print('max::::::: ', torch.max(outputs.data,1))
#         print("predicted:", predicted.shape)
        total += test_label.size(0)
        predicted_tmp = predicted.cpu().numpy()
        test_label_tmp = test_label.squeeze().data.cpu().numpy()
        correct += (predicted_tmp == test_label_tmp).sum()
        predicted_save = np.append(predicted_save, predicted_tmp)
        test_label_save = np.append(test_label_save, test_label_tmp)

    pd.DataFrame(predicted_save).to_csv(predicted_label_path, header=None, index= None)
    pd.DataFrame(test_label_save).to_csv(test_label_path, header=None, index= None)
    print('Accuracy of this fold is %d %%' % (100*correct/total))
    accuracy_training.append(100*correct/total)

average = sum(accuracy_training) / len(accuracy_training)
print('average accuracy = ', average)

fold 0


  0%|          | 0/100 [00:00<?, ?it/s]

using gpu # 0
epoch 0 :loss is 

  1%|          | 1/100 [00:01<01:44,  1.06s/it]

tensor(1.8096, device='cuda:0', dtype=torch.float64)
epoch 1 :loss is 

...

tensor(0.0004, device='cuda:0', dtype=torch.float64)
epoch 44 :loss is 










 45%|████▌     | 45/100 [00:46<00:56,  1.03s/it]

tensor(6.1203e-07, device='cuda:0', dtype=torch.float64)
epoch 45 :loss is 










 46%|████▌     | 46/100 [00:47<00:55,  1.04s/it]

tensor(5.3160e-10, device='cuda:0', dtype=torch.float64)
epoch 46 :loss is tensor(4.0059e-11, device='cuda:0', dtype=torch.float64)
training finished
Accuracy of this fold is 100 %
average accuracy =  91.00072979383324


In [6]:
print(accuracy_training)
print(np.mean(accuracy_training))
text_file = open(default_path+"/Training_Accuracy.txt","w")
text_file.write("Accuracy per fold: {} \nAverage: {}".format(accuracy_training, np.mean(accuracy_training)))
text_file.close()

[82.75862068965517, 96.42857142857143, 100.0, 67.85714285714286, 100.0, 100.0, 85.18518518518519, 92.5925925925926, 85.18518518518519, 100.0]
91.00072979383324


In [6]:
print(accuracy_training)
print(np.mean(accuracy_training))

[77.55102040816327, 80.20833333333333, 63.541666666666664, 77.08333333333333, 80.0, 75.78947368421052, 68.42105263157895, 81.05263157894737, 81.72043010752688, 76.34408602150538]
76.17120277652658


In [ ]:
train_root_path = default_path+ '/img_fold' + '0' + '/train'
img_name = '0'+'.png'
img_path = os.path.join(train_root_path, img_name)
imiji = io.imread(img_path)
print(imiji[0][:400])
img = np.empty(shape=(1, im_size * im_size))
img[0, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
print(img[0][:400])

# 